# Part 5: Receiving real data sensor from Bluetooth device


Now we are going to subscribe our sensors topic to start receiving real data. We are going to use a temporary certificate to access the MQTT topic during this workshop.

### Let's recover some data and stablish a MQTT connection with our own AWS Account & thing

In [ ]:
#Hour and minute when this started to create unique names
%store -r HM
#Home directory of the user running this notebook
%store -r home
#Private key file name, for AWS IoT authentication
%store -r private_key
#Certificate file name, for AWS IoT authentication
%store -r certificate_file
#Certificate authority file name, for AWS IoT authentication
%store -r ca_file
#Address to connect using MQTT
%store -r endpoint_address
#The name of the topic to send MQTT messages to
%store -r topic_name

from AWSIoTPythonSDK.MQTTLib import AWSIoTMQTTClient

client_id="NotebookCient11{}".format(HM)
my_mqtt = AWSIoTMQTTClient(client_id)
my_mqtt.configureEndpoint(endpoint_address, 8883)
my_mqtt.configureCredentials(ca_file, private_key, certificate_file)
my_mqtt.configureConnectDisconnectTimeout(10)
my_mqtt.connect()
my_mqtt

In [ ]:

! mkdir -p "{home}/aws_iot_hub"

other_certificate_file = "{}/aws_iot_hub/certificate-hub.pem".format(home)
other_public_key = "{}/aws_iot_hub/public-key-hub.pem".format(home)
other_private_key = "{}/aws_iot_hub/private-key-hub.pem".format(home)
other_ca_file = "{}/aws_iot_hub/ca.pem".format(home)

%store other_certificate_file
%store other_public_key
%store other_private_key
%store other_ca_file


In [ ]:
! wget https://s3.amazonaws.com/healthcare-workshop/certificate-hub.pem -O {other_certificate_file}
! wget https://s3.amazonaws.com/healthcare-workshop/public-key-hub.pem -O {other_public_key}
! wget https://s3.amazonaws.com/healthcare-workshop/private-key-hub.pem -O {other_private_key}
! wget https://www.symantec.com/content/en/us/enterprise/verisign/roots/VeriSign-Class%203-Public-Primary-Certification-Authority-G5.pem -O {other_ca_file}


Double check that you have the 4 files required for TLS mutual authentication used by AWS IoT
1. Private Key
1. Public Key
1. Certificate
1. Root CA

In [ ]:
! printf "Private Key {other_private_key} " && [ -f {other_private_key} ] && printf "found" || printf "not found"
! printf "\nPublic Key {other_public_key} " && [ -f {other_public_key} ] && printf "found" || printf "not found"
! printf "\nCertificate {other_certificate_file} " && [ -f {other_certificate_file} ] && printf "found" || printf "not found"
! printf "\nCertification Authority {other_ca_file} " && [ -f {other_ca_file} ] && printf "found" || printf "not found"

# Connecting to other AWS IoT Core

AWS IoT provides account-specific endpoints for your things to connect. We are using the instructor workshop endpoint at this time:

Ready to go! Create your MQTT client instance:

In [ ]:
from AWSIoTPythonSDK.MQTTLib import AWSIoTMQTTClient
other_endpoint_address = "a2enzbgohiblz2.iot.us-east-1.amazonaws.com"
%store other_endpoint_address
other_endpoint_address
client_id="NotebookCient22{}".format(HM)
other_mqtt = AWSIoTMQTTClient(client_id)
other_mqtt.configureEndpoint(other_endpoint_address, 8883)
other_mqtt.configureCredentials(other_ca_file, other_private_key, other_certificate_file)
other_mqtt.configureConnectDisconnectTimeout(10)
other_mqtt.connect()
other_mqtt

# Forward Data from other Thing / IoT Core to your Thing / IoT Core

In [ ]:
other_topic_name ="healthcare/data"
%store other_topic_name

subscribe_qos = 1
publish_qos = 0

def forward_message(client, userdata, message):
    print("\nReceived a new message: \n")
    print(message.payload.decode("utf-8") )
    my_mqtt.publish(topic_name, message.payload.decode("utf-8"), publish_qos)
    print("\n--------------\n")
    
other_mqtt.subscribe(other_topic_name, subscribe_qos, forward_message)

Now are going to forward the received message from Data Hub to our Thing in our account:

All working fine, done for now, disconnect.

In [ ]:
my_mqtt.disconnect()
other_mqtt.disconnect()


Now let's create a modern voice interface using [Alexa Nursing Skill](https://github.com/InternetOfHealthcare/aws-nursingskill).